In [1]:
!pip install pandas
!pip install numpy
!pip install -U scikit-learn
!python -m pip install -U matplotlib
!pip install --upgrade scikit-learn
!pip install spotipy --upgrade


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import json
import re
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [3]:
%matplotlib inline

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [6]:
spotify_df = pd.read_csv('tracks.csv')

In [7]:
spotify_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [8]:
spotify_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586672 entries, 0 to 586671
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                586672 non-null  object 
 1   name              586601 non-null  object 
 2   popularity        586672 non-null  int64  
 3   duration_ms       586672 non-null  int64  
 4   explicit          586672 non-null  int64  
 5   artists           586672 non-null  object 
 6   id_artists        586672 non-null  object 
 7   release_date      586672 non-null  object 
 8   danceability      586672 non-null  float64
 9   energy            586672 non-null  float64
 10  key               586672 non-null  int64  
 11  loudness          586672 non-null  float64
 12  mode              586672 non-null  int64  
 13  speechiness       586672 non-null  float64
 14  acousticness      586672 non-null  float64
 15  instrumentalness  586672 non-null  float64
 16  liveness          58

In [9]:
spotify_df.dtypes

id                   object
name                 object
popularity            int64
duration_ms           int64
explicit              int64
artists              object
id_artists           object
release_date         object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
dtype: object

In [10]:
spotify_df.shape # We have 586672 rows - 586672 datasets

(586672, 20)

In [11]:
spotify_df.size # Number of elements -  product represented by no. rows * no. cols

11733440

In [12]:
len(spotify_df) # 586672 rows

586672

In [13]:
# Sort the DataFrame by the 'popularity' column in descending order
spotify_df_sorted = spotify_df.sort_values(by='popularity', ascending=False)
spotify_df_sorted.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
93802,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),100,198082,1,"['Justin Bieber', 'Daniel Caesar', 'Giveon']","['1uNFoZAHBGtllmzznpCI3s', '20wkVLutqVOYrc0kxF...",2021-03-19,0.677,0.696,0,-6.181,1,0.1190,0.3210,0.000000,0.420,0.464,90.030,4
93803,7lPN2DXiMsVn7XUKtOW1CS,drivers license,99,242014,1,['Olivia Rodrigo'],['1McMsnEElThX1knmY4oliG'],2021-01-08,0.585,0.436,10,-8.761,1,0.0601,0.7210,0.000013,0.105,0.132,143.874,4
93804,3Ofmpyhv5UAQ70mENzB277,Astronaut In The Ocean,98,132780,0,['Masked Wolf'],['1uU7g3DNSbsu0QjSEqZtEd'],2021-01-06,0.778,0.695,4,-6.865,0,0.0913,0.1750,0.000000,0.150,0.472,149.996,4
92810,5QO79kh1waicV47BqGRL3g,Save Your Tears,97,215627,1,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2020-03-20,0.680,0.826,0,-5.487,1,0.0309,0.0212,0.000012,0.543,0.644,118.051,4
92811,6tDDoYIxWvMLTdKpjFkc1B,telepatía,97,160191,0,['Kali Uchis'],['1U1el3k54VvEUzo3ybLPlM'],2020-12-04,0.653,0.524,11,-9.016,0,0.0502,0.1120,0.000000,0.203,0.553,83.970,4


In [14]:
# Feature Extraction: Display only the 'name' and 'artist' columns
spotify_df_sorted_fe = spotify_df_sorted[['name', 'artists']]
spotify_df_sorted_fe.head()

,name,artists
93802,Peaches (feat. Daniel Caesar & Giveon),"['Justin Bieber', 'Daniel Caesar', 'Giveon']"
93803,drivers license,['Olivia Rodrigo']
93804,Astronaut In The Ocean,['Masked Wolf']
92810,Save Your Tears,['The Weeknd']
92811,telepatía,['Kali Uchis']


In [15]:
print(spotify_df_sorted['release_date'].unique())

['2021-03-19' '2021-01-08' '2021-01-06' ... '1950-10-30' '1949-09-01'
 '1949-11-27']


In [16]:
spotify_df_sorted['release_date'] = pd.to_datetime(spotify_df_sorted['release_date'], errors='coerce')

In [17]:
spotify_df_sorted_1990 = spotify_df_sorted[spotify_df_sorted['release_date'].dt.year > 1990]
spotify_df_sorted_1990.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
93802,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),100,198082,1,"['Justin Bieber', 'Daniel Caesar', 'Giveon']","['1uNFoZAHBGtllmzznpCI3s', '20wkVLutqVOYrc0kxF...",2021-03-19,0.677,0.696,0,-6.181,1,0.1190,0.3210,0.000000,0.420,0.464,90.030,4
93803,7lPN2DXiMsVn7XUKtOW1CS,drivers license,99,242014,1,['Olivia Rodrigo'],['1McMsnEElThX1knmY4oliG'],2021-01-08,0.585,0.436,10,-8.761,1,0.0601,0.7210,0.000013,0.105,0.132,143.874,4
93804,3Ofmpyhv5UAQ70mENzB277,Astronaut In The Ocean,98,132780,0,['Masked Wolf'],['1uU7g3DNSbsu0QjSEqZtEd'],2021-01-06,0.778,0.695,4,-6.865,0,0.0913,0.1750,0.000000,0.150,0.472,149.996,4
92810,5QO79kh1waicV47BqGRL3g,Save Your Tears,97,215627,1,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2020-03-20,0.680,0.826,0,-5.487,1,0.0309,0.0212,0.000012,0.543,0.644,118.051,4
92811,6tDDoYIxWvMLTdKpjFkc1B,telepatía,97,160191,0,['Kali Uchis'],['1U1el3k54VvEUzo3ybLPlM'],2020-12-04,0.653,0.524,11,-9.016,0,0.0502,0.1120,0.000000,0.203,0.553,83.970,4


In [18]:
spotify_df_sorted.isna().sum() # Since 71 is a significantly small number compared to 586672 total datasets, dropping these rows is an appropriate way to handle the data

id                       0
name                    71
popularity               0
duration_ms              0
explicit                 0
artists                  0
id_artists               0
release_date        138591
danceability             0
energy                   0
key                      0
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness         0
liveness                 0
valence                  0
tempo                    0
time_signature           0
dtype: int64

In [19]:
spotify_df_sorted = spotify_df_sorted[spotify_df_sorted['name'].isnull()]
spotify_df_sorted

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
226336,4iH7negBYMfj2z0wDNmgdx,NaN,28,264973,0,[''],['0LyfQWJT6nXafLPZqxe9Of'],1994-01-01,0.512,0.5780,0,-12.280,0,0.0299,0.0433,0.000064,0.5160,0.692,156.465,1
525238,1kR4gIb7nGxHPI3D2ifs59,NaN,26,289440,0,[''],['0LyfQWJT6nXafLPZqxe9Of'],1998-01-05,0.501,0.5830,7,-9.460,0,0.0605,0.6900,0.003960,0.0747,0.734,138.391,4
525265,4xyBq8t4nkUKH8s28j6ZoD,NaN,24,254000,0,[''],['0LyfQWJT6nXafLPZqxe9Of'],1998-01-05,0.390,0.4520,7,-9.900,1,0.0361,0.7370,0.000000,0.1110,0.662,94.630,4
520127,0hKA9A2JPtFdg0fiMhyjQD,NaN,6,194081,0,[''],['0LyfQWJT6nXafLPZqxe9Of'],1974-12-31,0.471,0.3690,4,-12.927,0,0.1460,0.9680,0.001100,0.1410,0.766,94.063,4
516861,1wLOAmEvTeTcEN6DpG6slJ,NaN,4,299640,0,[''],['0LyfQWJT6nXafLPZqxe9Of'],1961-03-06,0.449,0.2410,10,-13.926,0,0.0307,0.7970,0.000000,0.1220,0.560,134.091,4
517163,7EwXAFxCnQet9Vuhh0Kb3j,NaN,3,309507,0,[''],['0LyfQWJT6nXafLPZqxe9Of'],1962-02-01,0.500,0.7650,9,-4.892,0,0.1100,0.7320,0.000002,0.0734,0.854,148.179,3
517176,5QQHqUZqhbepZwwhie2BUH,NaN,3,235253,0,[''],['0LyfQWJT6nXafLPZqxe9Of'],1962-02-01,0.611,0.5670,7,-5.642,0,0.0473,0.8180,0.000003,0.4460,0.778,105.355,4
517185,2q1eryqN1DwnXOJtQ9C3fg,NaN,2,252947,0,[''],['0LyfQWJT6nXafLPZqxe9Of'],1962-02-01,0.469,0.6050,0,-4.393,1,0.0461,0.7510,0.000545,0.8080,0.768,148.457,3
517191,5SWgcuhJFoGIm9DmaqzDsH,NaN,2,264787,0,[''],['0LyfQWJT6nXafLPZqxe9Of'],1962-02-01,0.489,0.7360,4,-4.659,1,0.1890,0.7810,0.000002,0.1090,0.843,85.460,4
517206,6OH9mz9aFbGlbf74cBwYWD,NaN,2,209760,0,[''],['0LyfQWJT6nXafLPZqxe9Of'],1962-02-01,0.506,0.5980,7,-4.672,0,0.0635,0.7710,0.000001,0.6910,0.800,91.172,4


In [20]:
spotify_df_sorted_drop = spotify_df_sorted
spotify_df_sorted_drop.dropna(inplace=True)

In [21]:
spotify_df_sorted_drop.isna().sum()

id                  0
name                0
popularity          0
duration_ms         0
explicit            0
artists             0
id_artists          0
release_date        0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64

In [22]:
spotify_df_sorted['name'].fillna('Unknown', inplace=True)
# spotify_df['artists'].fillna('Unknown', inplace=True) # Apply this after converting artists name into a list (from strings)

In [23]:
print(spotify_df_sorted.index)

Index([], dtype='int64')


In [24]:
# Load the DataFrame from the uploaded file (assuming it's a CSV for the example)
import pandas as pd

# Since we don't have the actual CSV file, we will simulate the DataFrame
# We'll create a DataFrame with a range of indices that does NOT include 525238 to mimic the error
df = pd.DataFrame({
    'name': ['Song ' + str(i) for i in range(1000)]  # creating a small DataFrame for the example
})
df.index = range(500, 1500)  # setting index starting at 500 to 1499

# Now we will try to access the index 525238 to reproduce the KeyError
# And then we will implement the checks and fixes discussed

# Check if index 525238 exists in the DataFrame
index_exists = 525238 in df.index

# Print the result
index_exists


False

In [25]:
# try to access the row by its absolute position using iloc.
try:
    last_row = df.iloc[-1]
    access_by_position = True
except IndexError:
    access_by_position = False

# now reset the index and try to access a row with the new sequential index
df.reset_index(drop=True, inplace=True)
try:
    # acessing what is now the last row after reset, which would have been 525238 if it existed
    row_after_reset = df.iloc[-1]
    reset_index_access = True
except IndexError:
    reset_index_access = False

(access_by_position, reset_index_access)


(True, True)

In [26]:
nan_name2 = spotify_df_sorted[spotify_df_sorted['name'].isnull()]

In [27]:
nan_name2

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature


In [28]:
spotify_df_sorted_drop.isna().sum()

id                  0
name                0
popularity          0
duration_ms         0
explicit            0
artists             0
id_artists          0
release_date        0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64

In [29]:
spotify_df_sorted['artists_upd_v1'] = spotify_df_sorted['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))

In [30]:
spotify_df_sorted.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1


In [31]:
spotify_df_sorted_str = spotify_df_sorted[spotify_df_sorted['artists_upd_v1'].apply(lambda x: not x)]
spotify_df_sorted_str.head()

""


In [32]:
spotify_df_sorted_str.shape # 4739 datasets out of 586672 consists of apostrophes

(0, 0)

In [33]:
spotify_df_sorted['artists_upd_v2'] = spotify_df_sorted['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
spotify_df_sorted['artists_upd'] = np.where(spotify_df_sorted['artists_upd_v1'].apply(lambda x: not x), spotify_df_sorted['artists_upd_v2'], spotify_df_sorted['artists_upd_v1'] )

In [34]:
spotify_df_sorted_str2 = spotify_df_sorted[spotify_df_sorted['artists_upd_v2'].apply(lambda x: not x)]
spotify_df_sorted_str2.head()

""


In [35]:
spotify_df_sorted_str2.shape # 578380 data sets out of 586672 does not consists of apostrophes (4739 + 578380 = 582977, 586672 - 582977 = 3553 where did these dataset go?) - ignore for the moment since it is less than 1% of the data

(0, 0)

In [36]:
spotify_df_sorted_fe = spotify_df_sorted[['name', 'artists_upd']]
spotify_df_sorted_fe.head()

,name,artists_upd


In [37]:
id_list = spotify_df['id'].tolist()
len(id_list)

586672

In [38]:
# Convert to set to remove duplicates
unique_id = set(id_list)
len(unique_id) # We have ensured that all ids are unique

586672

In [39]:
spotify_df_sorted.sort_values(['artists','release_date'], ascending=False, inplace=True) # Determine whether the previous sorting is required - determine whether sorting by popularity or artist is more appropriate - i guess previous sorting is not required
spotify_df_sorted.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd


In [40]:
rows_with_nan = spotify_df_sorted[spotify_df_sorted['artists_upd'].isnull()]
rows_with_nan

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd


In [41]:
len(spotify_df_sorted['artists_upd'])

0

In [42]:
#len(spotify_df_sorted['artists_upd'][226336])

In [43]:
# Check for empty lists in the 'column_name' column
empty_lists = spotify_df_sorted[spotify_df_sorted['artists_upd'].apply(lambda x: isinstance(x, list) and len(x) == 1 and x[0] == '')]
empty_lists

""


In [44]:
len(empty_lists)

0

In [45]:
# Update 'artists_upd_v1' column with non-empty strings or 'Unknown'
spotify_df_sorted['artists_upd_v1'] = spotify_df_sorted['artists'].apply(lambda x: [name if name else 'Unknown' for name in re.findall(r"'([^']*)'", x)])

# Update 'artists_upd_v2' column with non-empty strings or 'Unknown'
spotify_df_sorted['artists_upd_v2'] = spotify_df_sorted['artists'].apply(lambda x: [name if name else 'Unknown' for name in re.findall('\"(.*?)\"', x)])

# Combine 'artists_upd_v1' and 'artists_upd_v2', replacing empty lists with 'Unknown'
spotify_df_sorted['artists_upd'] = np.where(
    spotify_df_sorted['artists_upd_v1'].apply(lambda x: not x),
    spotify_df_sorted['artists_upd_v2'],
    spotify_df_sorted['artists_upd_v1']
)

In [46]:
empty_lists = spotify_df_sorted[spotify_df_sorted['artists_upd'].apply(lambda x: isinstance(x, list) and len(x) == 1 and x[0] == '')]
empty_lists # we have successfuly replaced empty strings to "Unknown" in the artists_upd column

""


In [47]:
unknwon_lists = spotify_df_sorted[spotify_df_sorted['artists_upd'].apply(lambda x: isinstance(x, list) and x[0] == 'Unknown')] # Edit code! to ensure that "Unknown" replaces only valid rows in the dataset
unknwon_lists

""


In [48]:
# Check if id_y represents id_artists in tracks.csv
#type(spotify_df_sorted["id_artists"][13187])

In [49]:
# Function to clean and preprocess id_artists
def preprocess_genres(id_str):
    # Remove square brackets using str.replace()
    modified_string = id_str.replace("[", "").replace("]", "").replace("'", "").replace(" ", "_").replace(",_", " ") # consider using regular expression for improvement
    cleaned_id = modified_string.split()
    return cleaned_id

# Apply the preprocessing function to the 'genres' column
id_list_preprocessed = spotify_df_sorted['id_artists'].apply(preprocess_genres)
id_list_preprocessed

Series([], Name: id_artists, dtype: object)

In [50]:
spotify_df_sorted['id_artists_upd'] = id_list_preprocessed

In [51]:
peaches_songs = spotify_df_sorted[spotify_df_sorted['name'].str.contains('Peaches')]
peaches_songs.head(10) # We can observe that there are repetition in the dataset - remastered version. What kind of effect does it have when creating a recommendation system. Does it have a biase towards these songs? Should we fine-tune it accordingly

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,id_artists_upd


In [52]:
harry_songs = spotify_df_sorted[spotify_df_sorted['artists'].str.contains('Harry Styles', case=False)]
harry_songs.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,id_artists_upd


In [53]:
spotify_df_sorted[spotify_df_sorted['name']=='Adore You'] # If id_artist or name of artist and the name of the song is the same, then it is a duplicate - consider ways to handle this duplicates - take the average of the values?

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,id_artists_upd


In [54]:
justin_bieber_songs = spotify_df_sorted[spotify_df['artists'].str.contains('Justin Bieber', case=False)]
justin_bieber_songs

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,id_artists_upd


In [55]:
genre_data = pd.read_csv('artists.csv')
genre_data.head()

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0


In [56]:
genre_data.dtypes

id             object
followers     float64
genres         object
name           object
popularity      int64
dtype: object

In [57]:
genre_data['genres'].values[0]

'[]'

In [58]:
genre_data['genres'].values[0][0]

'['

In [59]:
genre_data['genres_upd'] = genre_data['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [60]:
genre_data['genres_upd'].values[0]

[]

In [61]:
spotify_df['artists_upd_v1'] = spotify_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))

In [62]:
spotify_df['artists'].values[0]

"['Uli']"

In [63]:
spotify_df['artists_upd_v1'].values[0][0]

'Uli'

In [64]:
spotify_df[spotify_df['artists_upd_v1'].apply(lambda x: not x)].head(5)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1
164,1xEEYhWxT4WhDQdxfPCT8D,Snake Rag,20,194533,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.708,0.361,0,-11.764,0,0.0441,0.994,0.883,0.103,0.902,105.695,4,[]
170,3rauXVLOOM5BlxWqUcDpkg,Chimes Blues,14,170827,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.546,0.189,0,-15.984,1,0.0581,0.996,0.908,0.339,0.554,80.318,4,[]
172,1UdqHVRFYMZKU2Q7xkLtYc,Pickin' On Your Baby,11,197493,0,"[""Clarence Williams' Blue Five""]",['6RuQvIr0t0otZHnAxXTGkm'],1923,0.520,0.153,0,-14.042,1,0.0440,0.995,0.131,0.353,0.319,102.937,4,[]
174,0Vl2DO5U6FjgBpzCtBN3OA,Everybody Loves My Baby,10,152507,0,"[""Clarence Williams' Blue Five""]",['6RuQvIr0t0otZHnAxXTGkm'],1923,0.514,0.193,2,-13.920,0,0.2380,0.996,0.199,0.248,0.665,180.674,4,[]
180,5SvyP1ZeJX1jA7AOZD08NA,Tears,10,187227,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.359,0.357,3,-11.810,1,0.0511,0.994,0.819,0.290,0.753,205.053,4,[]


In [65]:
spotify_df['artists_upd_v2'] = spotify_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
spotify_df['artists_upd'] = np.where(spotify_df['artists_upd_v1'].apply(lambda x: not x), spotify_df['artists_upd_v2'], spotify_df['artists_upd_v1'] )

In [66]:
# Fill missing values
spotify_df['artists_upd'] = spotify_df['artists_upd'].fillna('Unknown')
spotify_df['name'] = spotify_df['name'].fillna('Unknown')


In [67]:
# Drop rows with missing values in the relevant columns
spotify_df = spotify_df.dropna(subset=['artists_upd', 'name'])

In [68]:
spotify_df['artists_song'] = spotify_df.apply(lambda row: (row['artists_upd'][0] if row['artists_upd'] else 'Unknown') + row['name'], axis=1)

In [69]:
print(spotify_df['artists_song'].isna().sum())

0


In [70]:
spotify_df.sort_values(['artists_song','release_date'], ascending = False, inplace = True)

In [71]:
spotify_df[spotify_df['name']=='Adore You']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
86217,5AnCLGg35ziFOloEnXK4uu,Adore You,71,278747,0,['Miley Cyrus'],['5YGY8feqx7naU7z4HrwZM6'],2013-10-04,0.583,0.655,0,-5.407,1,0.0315,0.1110,0.000004,0.113,0.201,119.759,4,[Miley Cyrus],[],[Miley Cyrus],Miley CyrusAdore You
91884,3jjujdWJ72nww5eGnfs2E7,Adore You,88,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-13,0.676,0.771,8,-3.675,1,0.0483,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You
92524,1M4qEo4HE3PRaCOM7EXNJq,Adore You,74,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-06,0.676,0.771,8,-3.675,1,0.0483,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You


In [72]:
artists_exploded = spotify_df[['artists_upd','id']].explode('artists_upd')

In [73]:
print(artists_exploded.columns)
print(genre_data.columns)

Index(['artists_upd', 'id'], dtype='object')
Index(['id', 'followers', 'genres', 'name', 'popularity', 'genres_upd'], dtype='object')


In [74]:
artists_exploded_enriched = artists_exploded.merge(genre_data, how='left', left_on='artists_upd', right_on='name')
artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genres_upd.isnull()]

In [75]:
print(artists_exploded_enriched_nonnull.columns)

Index(['artists_upd', 'id_x', 'id_y', 'followers', 'genres', 'name',
       'popularity', 'genres_upd'],
      dtype='object')


In [76]:
artists_exploded_enriched_nonnull[artists_exploded_enriched_nonnull['id_x'] == '6KuQTIu1KoTTkLXKrwlLPV']

,artists_upd,id_x,id_y,followers,genres,name,popularity,genres_upd
216506,Robert Schumann,6KuQTIu1KoTTkLXKrwlLPV,2UqjDAXnDxejEyE0CzfUrZ,423826.0,"['classical', 'early romantic era', 'german ro...",Robert Schumann,64.0,"[classical, early_romantic_era, german_romanti..."
216507,Vladimir Horowitz,6KuQTIu1KoTTkLXKrwlLPV,4Ws5hSoABAwvGJ4LhHwHgq,92365.0,"['classical', 'classical performance', 'classi...",Vladimir Horowitz,54.0,"[classical, classical_performance, classical_p..."


In [77]:
artists_genres_merged = artists_exploded_enriched_nonnull.groupby('id_x')['genres_upd'].apply(list).reset_index()

In [78]:
artists_genres_merged['consolidates_genre_lists'] = artists_genres_merged['genres_upd'].apply(lambda x: list(set(list(itertools.chain.from_iterable(x)))))

In [79]:
artists_genres_merged.head()

,id_x,genres_upd,consolidates_genre_lists
0,0004Uy71ku11n3LMpuyf59,[[polish_rock]],[polish_rock]
1,000CSYu4rvd8cQ7JilfxhZ,"[[country_quebecois, rock_quebecois]]","[rock_quebecois, country_quebecois]"
2,000DsoWJKHdaUmhgcnpr8j,[[barnmusik]],[barnmusik]
3,000G1xMMuwxNHmwVsBdtj1,"[[candy_pop, new_wave, new_wave_pop, permanent...","[new_wave_pop, power_pop, candy_pop, new_wave,..."
4,000KblXP5csWFFFsD6smOy,"[[chamame, folclore_salteno, folklore_argentino]]","[chamame, folklore_argentino, folclore_salteno]"


In [80]:
print(spotify_df.columns)

Index(['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'artists_upd_v1',
       'artists_upd_v2', 'artists_upd', 'artists_song'],
      dtype='object')


In [81]:
spotify_df = spotify_df.merge(artists_genres_merged[['id_x', 'consolidates_genre_lists']], left_on='id', right_on='id_x', how='left')

In [82]:
spotify_df.drop(columns=['id_x'], inplace=True)

In [83]:
spotify_df.tail()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
586667,3YCC4oCzagcfx4s3fp3XFz,Bandolera,35,322926,1,"[""At' Fat"", 'Yemil', 'Boza', 'El Tachi']","['455JBMYIT1lWf1Djj4rDc7', '7g4cPtKxNx146qdaY9...",2020-11-17,0.502,0.814,5,-2.688,0,0.1330,0.524,0.00000,0.1510,0.6720,72.344,4,"[ Fat"", , , , , ]",[At' Fat],"[ Fat"", , , , , ]","Fat"", Bandolera",NaN
586668,5sr38tHMSPpaENiNcjUpFi,Cathrina,10,184560,0,"[""Henry D' Cruz"", ""Helen D'Souza""]","['3nH2WLSi1MhvlmJNkbLbIv', '4NR8lI9YoPWo5v2Kv6...",1976-01-01,0.661,0.677,1,-8.447,0,0.1160,0.683,0.00154,0.0361,0.8090,116.855,4,"[ Cruz"", ""Helen D]","[Henry D' Cruz, Helen D'Souza]","[ Cruz"", ""Helen D]","Cruz"", ""Helen DCathrina",NaN
586669,6Pu2leLWEfThVIqJilw6O9,The Passion of Our Lord according to St. Matth...,4,79027,0,"[""Boys' Choir of The Church of The Transfigura...","['47EwjzhfZKotjVtvlQFNFS', '5H155SiWINLkYRElqw...",1963,0.177,0.208,11,-22.988,0,0.0388,0.988,0.94600,0.0982,0.0587,132.053,3,"[ Choir of The Church of The Transfiguration"",...",[Boys' Choir of The Church of The Transfigurat...,"[ Choir of The Church of The Transfiguration"",...","Choir of The Church of The Transfiguration"", ...",NaN
586670,7f092ECp06XbFS6Ms5Yk6R,Mia Oraia Petalouda,26,100313,0,"[""Typaldos' Children's Choire""]",['7JJrBciIAQBPOBB5U6u49p'],2014-06-08,0.773,0.183,0,-9.370,1,0.0370,0.956,0.00000,0.1190,0.8640,79.950,4,[ Children],[Typaldos' Children's Choire],[ Children],ChildrenMia Oraia Petalouda,NaN
586671,1yw3fl6UGrxjq3BTeJh6xz,I Koukouvagia,28,96508,0,"[""Typaldos' Children's Choire""]",['7JJrBciIAQBPOBB5U6u49p'],2014-06-08,0.748,0.256,9,-10.177,1,0.0532,0.969,0.00000,0.1200,0.9240,179.898,4,[ Children],[Typaldos' Children's Choire],[ Children],ChildrenI Koukouvagia,NaN


In [84]:
genre_data.shape

(1162095, 6)

In [85]:
genre_data.size

6972570

In [86]:
genre_data.isna().sum() # Since 71 is a significantly small number compared to 586672 total datasets, dropping these rows is an appropriate way to handle the data

id             0
followers     11
genres         0
name           3
popularity     0
genres_upd     0
dtype: int64

In [87]:
nan_followers = genre_data[genre_data['followers'].isnull()] # Think of what the best method is to preprocess data with unknown song name and unknown artist name
nan_followers

,id,followers,genres,name,popularity,genres_upd
436666,7F71W80jaXFARK7hBjsDI2,NaN,['czech pop'],Marcell,36,[czech_pop]
436667,3MLHJz04KmEVzCTPclzkEm,NaN,['czech pop'],Niko,21,[czech_pop]
437264,0cqZsULDZdJTGA4Zqh8Ckv,NaN,[],Savzilla,0,[]
437265,0BuknWzKujyc9HfZ1V50Uk,NaN,[],Duck Doja,0,[]
439102,6ltU5gIDLmWNYaVNHnll5G,NaN,['mexican electronic'],Zofa,0,[mexican_electronic]
439103,7C9nWRMbRqpPUuKh2OEw9n,NaN,[],MHV,3,[]
439104,41c30F8zy5UCTSevbn0WfD,NaN,['mexican electronic'],Broadband Star,0,[mexican_electronic]
457848,1DK979aOesiZ4Vkus8txqu,NaN,[],AmorArtis Orchestra & Johannes Somary,2,[]
457849,2lr0R5vHGfI0C489h0r6qV,NaN,['dc indie'],Black Dog Prowl,16,[dc_indie]
457850,0xkSOIeyeTILNIOZKyFgaP,NaN,[],Band of the Fifteenth Field Artillery Regiment...,3,[]


In [88]:
len(nan_followers)

11

In [89]:
genre_data_sorted_followers = genre_data.sort_values(by='followers', ascending=False)
genre_data_sorted_followers.head()

,id,followers,genres,name,popularity,genres_upd
144138,6eUKZXaKkcviH0Ku9w2n3V,78900234.0,"['pop', 'uk pop']",Ed Sheeran,92,"[pop, uk_pop]"
144485,66CXWjxzNUsdJxJ2JdwvnR,61301006.0,"['pop', 'post-teen pop']",Ariana Grande,95,"[pop, post-teen_pop]"
313676,3TVXtAsR1Inumwj472S9r4,54416812.0,"['canadian hip hop', 'canadian pop', 'hip hop'...",Drake,98,"[canadian_hip_hop, canadian_pop, hip_hop, pop_..."
144481,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",Justin Bieber,100,"[canadian_pop, pop, post-teen_pop]"
144488,7dGJo4pcD2V6oG8kP0tJRR,43747833.0,"['detroit hip hop', 'hip hop', 'rap']",Eminem,94,"[detroit_hip_hop, hip_hop, rap]"


In [90]:
genre_data_sorted_popularity = genre_data.sort_values(by='popularity', ascending=False)
genre_data_sorted_popularity.head()

,id,followers,genres,name,popularity,genres_upd
144481,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",Justin Bieber,100,"[canadian_pop, pop, post-teen_pop]"
126338,06HL4z0CvFAxyc27GXpf02,38869193.0,"['pop', 'post-teen pop']",Taylor Swift,98,"[pop, post-teen_pop]"
313676,3TVXtAsR1Inumwj472S9r4,54416812.0,"['canadian hip hop', 'canadian pop', 'hip hop'...",Drake,98,"[canadian_hip_hop, canadian_pop, hip_hop, pop_..."
115489,4q3ewBCX7sLwd24euuV69X,32244734.0,"['latin', 'reggaeton', 'trap latino']",Bad Bunny,98,"[latin, reggaeton, trap_latino]"
144484,3Nrfpe0tUJi4K4DXYWgMUX,31623813.0,"['k-pop', 'k-pop boy group']",BTS,96,"[k-pop, k-pop_boy_group]"


In [91]:
# What are the differences between these three way sof sorting data. Which one is the most appropriate?
genre_data_sorted = genre_data.sort_values(by=['followers', 'popularity'], ascending=[False, False])
genre_data_sorted.head()

,id,followers,genres,name,popularity,genres_upd
144138,6eUKZXaKkcviH0Ku9w2n3V,78900234.0,"['pop', 'uk pop']",Ed Sheeran,92,"[pop, uk_pop]"
144485,66CXWjxzNUsdJxJ2JdwvnR,61301006.0,"['pop', 'post-teen pop']",Ariana Grande,95,"[pop, post-teen_pop]"
313676,3TVXtAsR1Inumwj472S9r4,54416812.0,"['canadian hip hop', 'canadian pop', 'hip hop'...",Drake,98,"[canadian_hip_hop, canadian_pop, hip_hop, pop_..."
144481,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",Justin Bieber,100,"[canadian_pop, pop, post-teen_pop]"
144488,7dGJo4pcD2V6oG8kP0tJRR,43747833.0,"['detroit hip hop', 'hip hop', 'rap']",Eminem,94,"[detroit_hip_hop, hip_hop, rap]"


In [92]:
# Display only the 'name' and 'artist' columns
genre_data_sorted_fe = genre_data_sorted[['name', 'followers', 'popularity']]
genre_data_sorted_fe.head()

,name,followers,popularity
144138,Ed Sheeran,78900234.0,92
144485,Ariana Grande,61301006.0,95
313676,Drake,54416812.0,98
144481,Justin Bieber,44606973.0,100
144488,Eminem,43747833.0,94


In [93]:
genre_data_sorted['genres'].dtype # if it returns 0, it suggests that the 'genres' column contains mixed data types, including integers, floats, strings, lists, or even NaN values.

dtype('O')

In [94]:
genre_data_sorted['genres'].apply(type).unique()

array([<class 'str'>], dtype=object)

In [95]:
genre_data_sorted['genres'].values[0]

"['pop', 'uk pop']"

In [96]:
genre_data_sorted['genres'].values[0][0] # Check whether "genre" is in a list format or a string format

'['

In [97]:
type(genre_data_sorted['genres'][0])

str

In [98]:
genre_data_sorted['genres'][0] # What is the difference between using a values() method?

'[]'

In [99]:
genre_data_sorted['genres'].head()

144138                                    ['pop', 'uk pop']
144485                             ['pop', 'post-teen pop']
313676    ['canadian hip hop', 'canadian pop', 'hip hop'...
144481             ['canadian pop', 'pop', 'post-teen pop']
144488                ['detroit hip hop', 'hip hop', 'rap']
Name: genres, dtype: object

In [100]:
spotify_df.isna().sum()

id                             0
name                           0
popularity                     0
duration_ms                    0
explicit                       0
artists                        0
id_artists                     0
release_date                   0
danceability                   0
energy                         0
key                            0
loudness                       0
mode                           0
speechiness                    0
acousticness                   0
instrumentalness               0
liveness                       0
valence                        0
tempo                          0
time_signature                 0
artists_upd_v1                 0
artists_upd_v2                 0
artists_upd                    0
artists_song                   0
consolidates_genre_lists    9563
dtype: int64

In [101]:
genre_data_sorted['genres'][105990]

'[]'

In [102]:
type(genre_data_sorted['genres'][105900])

str

In [103]:
strings_mask = genre_data_sorted['genres'].apply(lambda x: isinstance(x, str))

In [104]:
strings_variable = genre_data_sorted.loc[strings_mask, 'genres']

In [105]:
floats_variable = genre_data_sorted.loc[genre_data_sorted['genres'].apply(lambda x: isinstance(x, (int, float))), 'genres']

In [106]:
# Fill NaN values in 'genres' column with empty lists using apply and lambda
genre_data_sorted['genres'] = genre_data_sorted['genres'].apply(lambda x: x if isinstance(x, str) else "")

In [107]:
# Function to clean and preprocess genres
def preprocess_genres(genre_entry):
    # Remove square brackets using str.replace()
    modified_string = genre_entry.replace("[", "").replace("]", "").replace("'", "").replace(" ", "_").replace(",_", " ") # consider using regular expression for improvement
    cleaned_genres = modified_string.split()
    return cleaned_genres

# Apply the preprocessing function to the 'genres' column
genre_list_preprocessed = genre_data_sorted['genres'].apply(preprocess_genres)

# # Flatten the genres list and get unique genres
# unique_genres = set([genre for genres_list in df['genres'] for genre in genres_list])
# sorted_genres = sorted(unique_genres)

# # Print the unique genres
# print("Unique genres after preprocessing:")
# print(f"Genres: {unique_genres}\nLength: {len(unique_genres)}")
# print(f"Genres: {sorted_genres}\nLength: {len(sorted_genres)}")

In [108]:
type(genre_list_preprocessed[0]) # successful conversion to list!

list

In [109]:
genre_data_sorted['genres_upd'] = genre_list_preprocessed

In [111]:
genre_data_sorted = genre_data_sorted.drop('genres', axis=1)

In [112]:
artists_exploded = spotify_df_sorted[['artists_upd','id']].explode('artists_upd')
artists_exploded.tail(10)

,artists_upd,id


In [114]:
spotify_df["artists"][281020]

"['Les Cerfs-Volants']"

In [116]:
artists_exploded_enriched = artists_exploded.merge(genre_data_sorted, how = 'left', left_on = 'artists_upd',right_on = 'name') # Why are there a lot of NaN values - fix it
artists_exploded_enriched.tail(10) # The NaN values implies that the data cleansing was not executed appropriately - 3. Edit regular expressions: 1. "Unknown" values should be given to appropirate data 2. Make sure artists with same name are updated accordingly - regular expression should handle ' as well as " and \ appropriately. 3. Remove "," from the list

,artists_upd,id_x,id_y,followers,name,popularity,genres_upd


In [117]:
# Check how many rows consists of NaN values
artists_exploded_enriched.isna().sum()

artists_upd    0
id_x           0
id_y           0
followers      0
name           0
popularity     0
genres_upd     0
dtype: int64

In [118]:
artists_exploded_enriched.shape # approximately 23553 - 23567 data out of 763148 have NaN values - mishandling of data? this accounts for 3% of the dataset - drop these rows for now

(0, 7)

In [119]:
# Check if id_y represents id_artists in tracks.csv
id_y_df = spotify_df_sorted[spotify_df_sorted["id_artists"].apply(lambda x: "4AXpdpyOnye51mlXxsKESw" in x)]

In [120]:
artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genres_upd.isnull()] # Dropping NaN values handled here

In [121]:
len(artists_exploded_enriched_nonnull)

0

In [122]:
artists_exploded_enriched_nonnull[artists_exploded_enriched_nonnull['id_x'] =='6KuQTIu1KoTTkLXKrwlLPV'] # Why is there a song with same id_x

,artists_upd,id_x,id_y,followers,name,popularity,genres_upd


In [123]:
# Identify duplicates in id_x columns
duplicate_id_x= artists_exploded_enriched_nonnull[artists_exploded_enriched_nonnull.duplicated(subset='id_x', keep=False)]
duplicate_id_x

,artists_upd,id_x,id_y,followers,name,popularity,genres_upd


In [124]:
len(duplicate_id_x) # Why are there many duplicates? I thought we had ensured that the id in the spotify_df_sorted dataframe is unique - figure out why and handle it appropriately

0

In [125]:
artists_genres_consolidated = artists_exploded_enriched_nonnull.groupby('id_x')['genres_upd'].apply(list).reset_index()

In [126]:
artists_genres_consolidated['consolidates_genre_lists'] = artists_genres_consolidated['genres_upd'].apply(lambda x: list(set(list(itertools.chain.from_iterable(x)))))

In [127]:
artists_genres_consolidated.head()

,id_x,genres_upd,consolidates_genre_lists


In [129]:
print(spotify_df_sorted.columns)
print(artists_genres_consolidated.columns)

Index(['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'artists_upd_v1',
       'artists_upd_v2', 'artists_upd', 'id_artists_upd'],
      dtype='object')
Index(['id_x', 'genres_upd', 'consolidates_genre_lists'], dtype='object')


In [130]:
spotify_df_sorted = spotify_df_sorted.merge(artists_genres_consolidated[['id_x', 'consolidates_genre_lists']], left_on='id', right_on='id_x', how='left')

In [131]:
spotify_df_sorted.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,id_artists_upd,id_x,consolidates_genre_lists


In [132]:
float_cols = spotify_df_sorted.dtypes[spotify_df_sorted.dtypes == 'float64'].index.values

In [133]:
ohe_cols = 'popularity'

In [134]:
spotify_df['popularity'].describe()

count    586672.000000
mean         27.570053
std          18.370642
min           0.000000
25%          13.000000
50%          27.000000
75%          41.000000
max         100.000000
Name: popularity, dtype: float64

In [156]:
# create 5 point buckets for popularity
spotify_df['popularity_red'] = spotify_df['popularity'].apply(lambda x: int(x/5))

In [157]:
# tfidf can't handle nulls so fill any null values with an empty list
artists_genres_consolidated['consolidates_genre_lists'] = artists_genres_consolidated['consolidates_genre_lists'].apply(lambda d: d if isinstance(d, list) else [])

In [176]:
spotify_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists,popularity_red
0,3u1C6nWVRoP5F0w8gGrDL3,사랑의 미로,25,222380,0,['최진희'],['1NSrAf8XJYJVgAXKoxaMet'],1987-06-01,0.367,0.194,7,-19.057,1,0.0400,0.617,0.000006,0.1620,0.367,144.316,4,[최진희],[],[최진희],최진희사랑의 미로,[trot],5
1,1Mv4u308L16NZDZiD6HZCy,사랑은 힘든가봐,28,213440,0,['지수'],['4c9QIMfEbIIynuaswyxGx9'],2005-12-23,0.675,0.785,4,-5.026,0,0.0280,0.379,0.000000,0.3530,0.623,103.008,4,[지수],[],[지수],지수사랑은 힘든가봐,NoGenre,5
2,1jvoY322nxyKXq8OBhgmSY,어떡하죠,44,244360,0,['지선'],['2Mo9NQaNCFCWSR5CnlfmbN'],2011-10-13,0.606,0.341,0,-7.094,1,0.0513,0.779,0.000000,0.1440,0.294,135.667,4,[지선],[],[지선],지선어떡하죠,NoGenre,8
3,1KldYdWxVyDrSepd1NY2Qg,어떡하죠,39,195693,0,['지선'],['2Mo9NQaNCFCWSR5CnlfmbN'],2009-03-06,0.610,0.840,2,-4.543,1,0.0468,0.245,0.000000,0.1540,0.657,126.992,4,[지선],[],[지선],지선어떡하죠,NoGenre,7
4,2ghebdwe2pNXT4eL34T7pW,그아픔까지사랑한거야,32,237688,0,['조정현'],['2WTpsPucygbYRnCnoEUkJQ'],1989-06-15,0.447,0.215,10,-16.478,1,0.0272,0.568,0.000001,0.0649,0.177,71.979,4,[조정현],[],[조정현],조정현그아픔까지사랑한거야,NoGenre,6


In [177]:
# Check the first few entries to understand what they contain
print(spotify_df['consolidates_genre_lists'].head())

# Check if all entries in the column are lists
are_lists = spotify_df['consolidates_genre_lists'].apply(lambda x: isinstance(x, list))
if not are_lists.all():
    # If not all entries are lists, print out the problematic entries
    print("Entries that are not lists:", spotify_df[~are_lists]['consolidates_genre_lists'])

# Check if all elements in each list are strings
are_strings = df['consolidates_genre_lists'].apply(lambda x: all(isinstance(i, str) for i in x) if isinstance(x, list) else False)
if not are_strings.all():
    # If not all elements are strings, print out the problematic lists
    print("Lists with non-string elements:", spotify_df[~are_strings]['consolidates_genre_lists'])

# Fix the problematic entries before applying the TfidfVectorizer


0     [trot]
1    NoGenre
2    NoGenre
3    NoGenre
4    NoGenre
Name: consolidates_genre_lists, dtype: object
Entries that are not lists: 1         NoGenre
2         NoGenre
3         NoGenre
4         NoGenre
5         NoGenre
6         NoGenre
7         NoGenre
8         NoGenre
9         NoGenre
10        NoGenre
11        NoGenre
12        NoGenre
13        NoGenre
14        NoGenre
15        NoGenre
16        NoGenre
17        NoGenre
19        NoGenre
34        NoGenre
35        NoGenre
36        NoGenre
37        NoGenre
38        NoGenre
39        NoGenre
40        NoGenre
42        NoGenre
43        NoGenre
44        NoGenre
45        NoGenre
46        NoGenre
47        NoGenre
48        NoGenre
49        NoGenre
50        NoGenre
51        NoGenre
52        NoGenre
53        NoGenre
54        NoGenre
55        NoGenre
56        NoGenre
57        NoGenre
58        NoGenre
59        NoGenre
60        NoGenre
61        NoGenre
62        NoGenre
63        NoGenre
64        NoGen

KeyError: 'consolidates_genre_lists'

In [182]:
# Replace NaN values with a placeholder string (e.g., "NoGenre")
spotify_df['consolidates_genre_lists'] = spotify_df['consolidates_genre_lists'].apply(lambda x: x if isinstance(x, list) and x != [] else "NoGenre")

# Now that NaN values are handled, let's check if all elements in each list are strings
are_strings = spotify_df['consolidates_genre_lists'].apply(lambda x: all(isinstance(i, str) for i in x) if isinstance(x, list) else True)

# If there are still lists with non-string elements, print them out
if not are_strings.all():
    print("Lists with non-string elements:", spotify_df[~are_strings]['consolidates_genre_lists'])

# Assuming all entries are now lists of strings, you can proceed with the TfidfVectorizer

In [183]:
#simple function to create OHE features
#this gets passed later on
def ohe_prep(df, column, new_name): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [184]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD

def create_feature_set(df, float_cols, n_components=500, max_features=10000):
    """
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters:
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled
        n_components (int): Number of components for dimensionality reduction
        max_features (int): Max number of terms to consider in the TF-IDF vectorization

    Returns:
        final: final set of features
    """

    # Replace NaN values with a placeholder string (e.g., "NoGenre")
    df['consolidates_genre_lists'] = df['consolidates_genre_lists'].apply(lambda x: x if isinstance(x, list) and x != [] else "NoGenre")

    # Using TfidfVectorizer with a limited number of features
    tfidf = TfidfVectorizer(max_features=max_features)
    tfidf_matrix = tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))

    # Use TruncatedSVD to reduce dimensionality
    svd = TruncatedSVD(n_components=n_components)
    tfidf_reduced = svd.fit_transform(tfidf_matrix)

    # Convert to DataFrame
    genre_df = pd.DataFrame(tfidf_reduced)
    genre_df.columns = ['genre_svd' + "|" + str(i) for i in range(tfidf_reduced.shape[1])]
    
    # One-hot encoding
    year_ohe = ohe_prep(df, 'year', 'year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red', 'pop') * 0.15

    # Scale float columns
    floats = df[float_cols].reset_index(drop=True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns=floats.columns) * 0.2

    # Concatenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis=1)

    # Add song id
    final['id'] = df['id'].values

    return final


In [ ]:
complete_feature_set = create_feature_set(spotify_df, float_cols=float_cols)#.mean(axis = 0)

In [ ]:
# Check the first few entries of the 'consolidates_genre_lists' column
print(artists_genres_consolidated['consolidates_genre_lists'].head())

In [ ]:
# Check for any non-empty lists in the 'consolidates_genre_lists' column
non_empty_genre_lists = artists_genres_consolidated['consolidates_genre_lists'].apply(lambda x: len(x) > 0)
print("Any non-empty genre lists:", non_empty_genre_lists.any())

In [ ]:
# If there are no non-empty genre lists, this will print the number of empty lists
print("Number of empty genre lists:", (~non_empty_genre_lists).sum())

In [ ]:
complete_feature_set = create_feature_set(artists_genres_consolidated, float_cols=float_cols)#.mean(axis = 0)

In [ ]:
# Check the number of rows in the dataframe
print("Number of rows:", len(artists_genres_consolidated))

# If the DataFrame is not empty, check the first few entries of the 'consolidates_genre_lists' column
if len(artists_genres_consolidated) > 0:
    print(artists_genres_consolidated['consolidates_genre_lists'].head())


In [ ]:
complete_feature_set.head()

In [ ]:
#client id and secret for my application
client_id = 5b6e2092b8df42b0802198df0a9972e3
client_secret= 1e9a6159d57d45a890bfd00f412f8529

In [ ]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [ ]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='http://localhost:8000/')

In [ ]:
sp = spotipy.Spotify(auth=token)

In [ ]:
#gather playlist names and images. 
#images aren't going to be used until I start building a UI
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [ ]:
id_name

In [ ]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram
        
    Returns: 
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """
    
    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [ ]:
id_name

In [ ]:
playlist_EDM = create_necessary_outputs('EDM', id_name,spotify_df)
#playlist_chill = create_necessary_outputs('chill',id_name, spotify_df)
#playlist_classical = create_necessary_outputs('Epic Classical',id_name, spotify_df)

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

def visualize_songs(df):
    """ 
    Visualize cover art of the songs in the inputted dataframe

    Parameters: 
        df (pandas dataframe): Playlist Dataframe
    """
    
    temp = df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5
    
    for i, url in enumerate(temp):
        plt.subplot(len(temp) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(df['name'].values[i], fontsize = 12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [ ]:
playlist_EDM

In [ ]:
visualize_songs(playlist_EDM)

In [ ]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        
    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-4].columns)
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    #playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']
    
    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [ ]:
complete_feature_set_playlist_vector_EDM, complete_feature_set_nonplaylist_EDM = generate_playlist_feature(complete_feature_set, playlist_EDM, 1.09)
#complete_feature_set_playlist_vector_chill, complete_feature_set_nonplaylist_chill = generate_playlist_feature(complete_feature_set, playlist_chill, 1.09)

In [ ]:
complete_feature_set_playlist_vector_EDM.shape

In [ ]:
from IPython.display import Image
Image("/Users/thakm004/Documents/Spotify/cosine_sim_2.png")

In [ ]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [ ]:
edm_top40 = generate_playlist_recos(spotify_df, complete_feature_set_playlist_vector_EDM, complete_feature_set_nonplaylist_EDM)

In [ ]:
edm_top40

In [ ]:
visualize_songs(edm_top40)

In [ ]:
chill_top40 = generate_playlist_recos(spotify_df, complete_feature_set_playlist_vector_chill, complete_feature_set_nonplaylist_chill)